In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import urlopen
import json
import os
from json import loads
import plotly.express as px
from dash import Dash, dcc, html, Input, Output


Helper functions to search for school

In [2]:
school_df = pd.read_csv("schoolname_UNITID.csv")

def search_school(school_name):
    # Case-insensitive search for school name
    result = school_df[school_df["inst_name"].str.contains(school_name, case=False, na=False)]
    if result.empty:
        print(f"No matches found for '{school_name}'.")
    return result

def get_school_name(unitid):
    # Fetch school directory data
    url = f"https://educationdata.urban.org/api/v1/college-university/ipeds/directory/2020/?unitid={unitid}"
    response = urlopen(url)
    data = loads(response.read())
    
    results = data.get("results", [])
    if results:
        school_name = results[0].get("inst_name", "Unknown")
        return school_name
    else:
        return f"No school found for unitid {unitid}"

def search_school_by_state(state_abbr):
    """Search for schools in a given state (by abbreviation) and return a DataFrame with unitid and school name.
    """
    url = "https://educationdata.urban.org/api/v1/college-university/ipeds/directory/2020/"
    
    try:
        response = urlopen(url)
        data = loads(response.read())
        results = data.get("results", [])
        
        if not results:
            print(f"No schools found.")
            return pd.DataFrame()
        
        # Filter results manually based on state_abbr
        filtered_results = [
            {"unitid": result.get("unitid"), "school_name": result.get("inst_name", "Unknown")}
            for result in results if result.get("state_abbr") == state_abbr.upper()
        ]
        
        if not filtered_results:
            print(f"No schools found in {state_abbr}.")
            return pd.DataFrame()
        
        return pd.DataFrame(filtered_results)
    
    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

In [ ]:
def create_variable_mapping(excel_path, sheet_name="varlist"):
    """
    Reads an Excel file from the 'varlist' sheet and extracts 'varname' and 'varTitle' columns to create a mapping.
    """
    df = pd.read_excel(excel_path, sheet_name=sheet_name)

    if "varname" not in df.columns or "varTitle" not in df.columns:
        raise ValueError("Required columns 'varname' and 'varTitle' not found in the Excel file.")

    mapping = {row["varname"]: row["varTitle"].replace(" ", "_") for _, row in df.iterrows()}
    return mapping


In [9]:
mapping = create_variable_mapping("adm\\adm2023_dict.xlsx")
print(mapping)

{'UNITID': 'Unique identification number of the institution', 'ADMCON1': 'Secondary school GPA', 'ADMCON2': 'Secondary school rank', 'ADMCON3': 'Secondary school record', 'ADMCON4': 'Completion of college-preparatory program', 'ADMCON5': 'Recommendations', 'ADMCON6': 'Formal demonstration of competencies', 'ADMCON7': 'Admission test scores', 'ADMCON8': 'English Proficiency Test', 'ADMCON9': 'Other Test (Wonderlic, WISC-III, etc.)', 'ADMCON10': 'Work experience', 'ADMCON11': 'Personal statement or essay', 'ADMCON12': 'Legacy status', 'APPLCN': 'Applicants total', 'APPLCNM': 'Applicants men', 'APPLCNW': 'Applicants women', 'APPLCNAN': 'Applicants another gender', 'APPLCNUN': 'Applicants gender unknown', 'ADMSSN': 'Admissions total', 'ADMSSNM': 'Admissions men', 'ADMSSNW': 'Admissions women', 'ADMSSNAN': 'Admissions another gender', 'ADMSSNUN': 'Admissions gender unknown', 'ENRLT': 'Enrolled total', 'ENRLM': 'Enrolled  men', 'ENRLW': 'Enrolled  women', 'ENRLAN': 'Enrolled another gender',